In [1]:
# import packages
!pip install faker
!pip install names
!pip uninstall -y dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl
!pip install dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl

import json
import pandas as pd
import faker
import os
from IPython.display import display
import requests
import random
import ipywidgets as wid
import numpy as np

Found existing installation: OpenSpecimenAPIconnector 0.9.1
Uninstalling OpenSpecimenAPIconnector-0.9.1:
  Successfully uninstalled OpenSpecimenAPIconnector-0.9.1
Processing ./dist/OpenSpecimenAPIconnector-0.9.1-py3-none-any.whl


In [2]:
import OpenSpecimenAPIconnector as OSconn
import OpenSpecimenAPIconnector.os_core as os_core
import OpenSpecimenAPIconnector.os_util as os_util
import OpenSpecimenAPIconnector.mg_core as mg_core
import OpenSpecimenAPIconnector.mg_util as mg_util

In [3]:
def csv_download_link(df, csv_file_name, delete_prompt=False):
    """Display a download link to load a data frame as csv from within a Jupyter notebook"""
    from IPython.core.display import display, HTML
    link = "http://localhost:8888/edit/work/" + csv_file_name
    df.to_csv(csv_file_name, index=False)
    text = "Download/Edit " + csv_file_name
    display(HTML("<a href={} target=_blank>{}</a>".format(link, text)))
   
    if delete_prompt:
        a = input('Press enter to delete the file after you have downloaded it.')
        import os
        os.remove(csv_file_name)

In [4]:
ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

def common_filtering(year, purpose):
    output.clear_output()
    
    if (year == ALL) & (purpose == ALL):
        common_filter = df_london
    elif (year == ALL):
        common_filter = df_london[df_london.purpose == purpose]
    elif (purpose == ALL):
        common_filter = df_london[df_london.year == year]
    else:
        common_filter = df_london[(df_london.year == year) & 
                                  (df_london.purpose == purpose)]
    
    with output:
        display(common_filter)

def dropdown_year_eventhandler(change):
    common_filtering(change.new, dropdown_purpose.value)

def dropdown_purpose_eventhandler(change):
    common_filtering(dropdown_year.value, change.new)

In [5]:
def multi_checkbox_widget(options_dict):
    """ Widget with a search field and lots of checkboxes """
    search_widget = wid.Text()
    output_widget = wid.Output()
    options = [x for x in options_dict.values()]
    options_layout = wid.Layout(
        overflow='auto',
        border='1px solid black',
        width='300px',
        height='300px',
        flex_flow='column',
        display='flex'
    )
    
    #selected_widget = wid.Box(children=[options[0]])
    options_widget = wid.VBox(options, layout=options_layout)
    #left_widget = wid.VBox(search_widget, selected_widget)
    multi_select = wid.VBox([search_widget, options_widget])

    @output_widget.capture()
    def on_checkbox_change(change):
        
        selected_recipe = change["owner"].description
        #print(options_widget.children)
        #selected_item = wid.Button(description = change["new"])
        #selected_widget.children = [] #selected_widget.children + [selected_item]
        options_widget.children = sorted([x for x in options_widget.children], key = lambda x: x.value, reverse = True)
        
    for checkbox in options:
        checkbox.observe(on_checkbox_change, names="value")

    # Wire the search field to the checkboxes
    @output_widget.capture()
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = sorted(options, key = lambda x: x.value, reverse = True)
        else:
            # Filter by search field using difflib.
            #close_matches = difflib.get_close_matches(search_input, list(options_dict.keys()), cutoff=0.0)
            close_matches = [x for x in list(options_dict.keys()) if str.lower(search_input.strip('')) in str.lower(x)]
            new_options = sorted(
                [x for x in options if x.description in close_matches], 
                key = lambda x: x.value, reverse = True
            ) #[options_dict[x] for x in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    display(output_widget)
    return multi_select

In [6]:
def display_search_select_box(cps_dict):
    descriptions = [key for key in  cps_dict.keys()]

    options_dict = {
        x: wid.Checkbox(
            description=x, 
            value=False,
            style={"description_width":"0px"}
        ) for x in descriptions
    }

    def f(**args):

        results = [key for key, value in args.items() if value]
        display(results)

    ui = multi_checkbox_widget(options_dict)
    out = wid.interactive_output(f, options_dict)
    display(wid.HBox([ui, out]))
    return ui

In [7]:
# The URL of the Openspecimen
print('Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter')
base_url= 'http://biobank-7-2.silicolab.bibbox.org' 
base_url+='/openspecimen/rest/ng'
print('The whole URL for API Calls is: ' + base_url)

# Loginname and Pasword of the API-User, recommended is an extra API User
print('Loginname of the user:')
loginname='admin'#input() 
print('Password of the user:')
password='Login@123'#input()

auth = (loginname, password)

OSconn.config_manager.set_login(base_url, auth)
print(OSconn.config_manager.get_url())
print(OSconn.config_manager.get_auth())


Fill in Openspecimen url (http(s)://<IP-Adress>:<port>) and press enter
The whole URL for API Calls is: http://biobank-7-2.silicolab.bibbox.org/openspecimen/rest/ng
Loginname of the user:
Password of the user:
http://biobank-7-2.silicolab.bibbox.org/openspecimen/rest/ng
('admin', 'Login@123')


The next step is to choose the entity which will be created or updated. The following schemas are in Openspecimen available: (Name, openspecimen entity). One have to write the openspecimen entity. ('Collection Protocol', 'cp') ('Collection Protocol Registration', 'cpr') ('User', 'user') ('User Roles', 'userRoles') ('Site', 'site') ('Shipments', 'shipment') ('Institute', 'institute') ('Distribution Protocol Requirement', 'dpRequirement') ('Distribution Protocol', 'distributionProtocol') ('Distribution Order', 'distributionOrder') ('Containers', 'storageContainer') ('Container Type', 'storagecontainerType') ('Container Shipment', 'containerShipment') ('Collection Protocol Event', 'cpe') ('Master Specimen', 'masterSpecimen') ('Participant', 'participant') ('Specimen Requirement', 'sr') ('Visit', 'visit') ('Specimen Aliquot', 'specimenAliquot') ('Specimen Derivate', 'specimenDerivative') ('Specimen Disposal', 'specimenDisposal') ('Consent', 'consent')

In [8]:
schema=wid.Dropdown(
    options= [(), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collection Protocol Registration', 'cpr'),
              ('User', 'user'), ('User Roles', 'userRoles'), ('Site', 'site'), ('Shipments', 'shipment'), 
              ('Institute', 'institute'), ('Distribution Protocol Requirement', 'dpRequirement'), 
              ('Distribution Protocol', 'distributionProtocol'), ('Distribution Order', 'distributionOrder'), 
              ('Containers', 'storageContainer'), ('Container Type', 'storagecontainerType'),
              ('Container Shipment', 'containerShipment'), ('Collection Protocol Event', 'cpe'),
              ('Master Specimen', 'masterSpecimen'), ('Participant', 'participant'), ('Specimen Requirement', 'sr'),
              ('Visit', 'visit'), ('Specimen Aliquot', 'specimenAliquot'), 
              ('Specimen Derivate', 'specimenDerivative'),('Specimen Disposal', 'specimenDisposal'),
              ('Consent', 'consent')],
    value=(),
    description='Entity:')

operation = wid.Dropdown(
    options = ["CREATE",  'UPDATE'],
              value="CREATE",
              description='Operation:')

sel_schema = schema.value
def change_value_schema(change):
    global sel_schema
    if change.new:
        sel_schema = change.new[1]
    else: sel_schema = None

op = operation.value
def change_value_op(change):
    global op
    if change.new:
        op = change.new
    else: op = None

schema.observe(change_value_schema, names="value")
operation.observe(change_value_op, names="value")

print('Choose your Entity:')
display(schema)
print('Choose Operation:')
display(operation)

Choose your Entity:


Dropdown(description='Entity:', options=((), ('Collection Protocol', 'cp'), ('Specimen', 'specimen'), ('Collec…

Choose Operation:


Dropdown(description='Operation:', options=('CREATE', 'UPDATE'), value='CREATE')

In [10]:
print("Selected Schema: ", sel_schema)
print("Selected Operation: ", op)

#initialize csv_bulk, bulk_operations and CSV_exporter
csv_files= os_core.csv_bulk(base_url,(loginname, password))
bulk_op= os_util.bulk_operations(base_url, (loginname, password))
csv_export = os_util.Export_OP(base_url, (loginname, password))

base_url= 'http://biobank-7-1.silicolab.bibbox.org' 
base_url+='/openspecimen/rest/ng'

OSconn.config_manager.set_login(base_url, auth)
csv_files= os_core.csv_bulk(base_url,(loginname, password))
bulk_op= os_util.bulk_operations(base_url, (loginname, password))
csv_export = os_util.Export_OP(base_url, (loginname, password))


Selected Schema:  ()
Selected Operation:  CREATE
http://biobank-7-2.silicolab.bibbox.org/openspecimen/rest/ng ('admin', 'Login@123')
http://biobank-7-2.silicolab.bibbox.org/openspecimen/rest/ng ('admin', 'Login@123')
http://biobank-7-1.silicolab.bibbox.org/openspecimen/rest/ng ('admin', 'Login@123')
http://biobank-7-1.silicolab.bibbox.org/openspecimen/rest/ng ('admin', 'Login@123')


In [10]:
# get necessary cp information
cp_tools = os_core.collection_protocol(base_url2, (loginname, password))

# get cp info and wirte a dict in order to simplify the api output
cps = cp_tools.get_all_collection_protocols()
cps_dict = {}
for i, item in enumerate(cps):
    # print("Cp Title:")
    # print(item["title"])
    # print("Cp ID: ",item["id"], "\n")
    cps_dict[item["title"]] = item["id"]

In [11]:
# choose target CP / Check if matching is correct; ui is needed for output

ui = display_search_select_box(cps_dict)

Output()

In [22]:
# get selected cp_ids
cp_ids = []
selected_options = [widget.description for widget in ui.children[1].children if widget.value]
for item in selected_options:
    cp_ids.append(cps_dict[item])

print("CP ids selected for further usage: ", cp_ids)
print()

# Download Template file to see available fields 
# CP Registration CSV are downloaded for the aggregtation example 
# in order to get participant age data for aggregating age fields in the given example:
export_cps_csv = []
export_cps_def = []
for i, cp_id in enumerate(cp_ids):
    # Example for downloading tempalte csv for differen entities 
    if i == 0:
        template = csv_files.get_template(schemaname=sel_schema)
        template_df =  pd.DataFrame(template)
        # print("Template CSV of {}:".format(sel_schema))
        # display(template_df)
    
    export_cps_csv.append(csv_export.export_file(sel_schema, cp_id))
    # display one of the downladed csv for the given entity
    if i == 0:
        print("Example CSV of {} with data:".format(sel_schema))
        display(export_cps_csv[-1])
    export_cps_def.append(cp_tools.get_cp_def(cp_id))

CP ids selected for further usage:  [17, 18]

Template CSV of cpr:


,Identifier,CP Short Title,PPID,Registration Date,Registration Site,External Subject ID,Activity Status,Reason,First Name,Last Name,...,Race#2,Vital Status,Ethnicity#1,Ethnicity#2,National ID,eMPI,PMI#1#Site Name,PMI#1#MRN,PMI#2#Site Name,PMI#2#MRN


Example CSV of cpr with data:


,Identifier,CP Short Title,PPID,Registration Date,Registration Site,External Subject ID,Activity Status,Reason,First Name,Last Name,...,Date Of Birth,Death Date,Gender,Race#1,Vital Status,Ethnicity#1,National ID,eMPI,PMI#1#Site Name,PMI#1#MRN
0,1,HBOOCA,71b0ffce-7e03-46da-97b4-65bed82d369f,17-02-2019,NaN,NaN,Active,NaN,Diana,F?reder,...,22-03-1932,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
1,2,HBOOCA,31e87180-6822-47c9-9530-1a9602217d2a,10-05-2011,NaN,NaN,Active,NaN,Yasmina,Walter,...,25-06-2009,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
2,3,HBOOCA,d6d2b04e-e099-4b7e-99f7-a66f7cdc3365,09-01-2012,NaN,NaN,Active,NaN,Jovana,Mairhofer,...,24-10-1937,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
3,4,HBOOCA,3951c14d-27cb-4a1f-8c50-9d9b7dbde9e4,16-11-2014,NaN,NaN,Active,NaN,Martha,Gr?nwald,...,06-03-1978,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
4,5,HBOOCA,775d7a91-df27-4f7d-b582-5be3cb6eb53e,02-01-2020,NaN,NaN,Active,NaN,Mariana,Wiesinger,...,02-08-2014,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3309,3310,HBOOCA,a99c2e45-508e-480a-8093-a113bdcdfde8,12-09-2013,NaN,NaN,Active,NaN,Lars,Grassl,...,10-11-1951,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
3310,3311,HBOOCA,ac5f0657-1cfd-4f7b-9499-750f8ccc0b48,22-02-2016,NaN,NaN,Active,NaN,Gloria,Foidl,...,16-08-1964,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
3311,3312,HBOOCA,37dca81f-4c53-4ebe-98f2-4662e439a2cf,26-03-2016,NaN,NaN,Active,NaN,Benedikt,Kober,...,31-07-1956,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN
3312,3313,HBOOCA,9526c7bc-1f2d-4c9f-8287-fa948d7d26be,12-01-2020,NaN,NaN,Active,NaN,Melina,Edlinger,...,27-06-1938,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN


In [23]:
# in case of multiple targets; just an example for Collection Protocolls exists do far since additional 
# fields are attached there. Also the csv export for Collection Protocolls does not work since 
# there is no generator. Should be there in latest version of OpenSpecimen
if sel_schema == "cpr":
    for h, cp in enumerate(export_cps_csv):
        age_array = []
        for date in cp["Date Of Birth"]:
            age_array.append(2020 - float(date.split("-")[-1]))
        age_array = np.array(age_array)
        print("Age high: ",np.max(age_array))
        print("Age low: ", np.min(age_array))
        print("Age mean: ", age_array.mean())
        cp_data = export_cps_def[h]
        for i, item in enumerate(cp_data["extensionDetail"]["attrs"]):
            if cp_data["extensionDetail"]["attrs"][i]["name"] == "AgeLow":
                cp_data["extensionDetail"]["attrs"][i]["value"] = str(int(np.min(age_array)))
                cp_data["extensionDetail"]["attrs"][i]["displayValue"] = str(np.min(age_array))
            elif cp_data["extensionDetail"]["attrs"][i]["name"] == "AgeHigh":
                cp_data["extensionDetail"]["attrs"][i]["value"] = str(int(np.max(age_array)))
                cp_data["extensionDetail"]["attrs"][i]["displayValue"] = str(np.max(age_array))
        # print("Current cp update:") 
        # print(cp["extensionDetail"]["attrs"])
        # input()
else:
    print("In this Demo the only fields to aggregate are attached to Collection Protocols")

Age high:  89.0
Age low:  5.0
Age mean:  46.87386843693422
Age high:  89.0
Age low:  5.0
Age mean:  47.277190332326285
